# Pesquisa sobre óbitos da COVID por número de doses recebidas

In [10]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import sqlite3
import datetime

# CONFIGURAÇÃO BÁSICA:
# Nome do arquivo CSV para análise
NOME_DO_ARQUIVO_CSV = "INFLUD22-25-07-2022.csv"

# descomentar para visualizar toda tabela
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Chaves e Descrições estão disponíveis no PDFs na pasta /docs

        docs/dicionario_de_dados_srag_hosp_DD_MM_AAAA.pdf

## Caso queira converter para banco de dados  

SQLite : importar usando [DB Browser for SQLite](https://sqlitebrowser.org/dl/)  

Comentar a linha:


        dataframe = pd.read_csv(NOME_DO_ARQUIVO_CSV, sep=";", decimal=",", encoding="utf8", low_memory=False)


E descomentar as linhas:  


        con = sqlite3.connect("database.sqlite")
        dataframe = pd.read_sql_query("SELECT * from tabelas", con)


In [11]:
dataframe =  pd.read_csv(NOME_DO_ARQUIVO_CSV, sep=";", decimal=",", encoding="utf8", low_memory=False)
# con = sqlite3.connect("database.sqlite")
# dataframe = pd.read_sql_query("SELECT * from tabelas", con)

# converter data para datetime object
dataframe["DT_NOTIFIC"] = pd.to_datetime(dataframe["DT_NOTIFIC"], dayfirst=True)
dataframe["DT_SIN_PRI"] = pd.to_datetime(dataframe["DT_SIN_PRI"], dayfirst=True)
df = dataframe
# converter data dd/mm/aaaa para datetime object
df["DOSE_1_COV"] = pd.to_datetime(df["DOSE_1_COV"], dayfirst=True, errors='coerce')
df["DOSE_2_COV"] = pd.to_datetime(df["DOSE_2_COV"], dayfirst=True, errors='coerce')
df["DOSE_REF"] = pd.to_datetime(df["DOSE_REF"], dayfirst=True, errors='coerce')
df["DT_NASC"] = pd.to_datetime(df["DT_NASC"], dayfirst=True, errors='coerce')

## Filtrar COVID como causa do óbito

In [12]:
# docs/dicionario_de_dados_srag_hosp_DD_MM_AAAA.pdf
# EVOLUCAO 
#   1 - Cura
#   2 - Óbito
#   3 - Óbito por outras causas
#   9 - Ignorado
# CLASSI_FIN
#   1 - SRAG por influenza
#   2 - SRAG por outro vírus respiratório
#   3 - SRAG por outro agente etiológico, qual:
#   4 - SRAG não especificado
#   5 - SRAG por covid-19

df_obito_por_covid = df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5)]

# deixar apenas colunas sobre vacinação para facilitar visualização
df_obito_por_covid = df_obito_por_covid.loc[:, df_obito_por_covid.columns.intersection(["DOSE_1_COV","DOSE_2_COV","DOSE_REF"])]
print("total mortes por COVID:", len(df_obito_por_covid))

total mortes por COVID: 46690


## Somar número de doses de vacina recebida
somar as doses aplicadas baseado na disponibilidade da data da aplicação da dose

In [13]:
df_obito_por_covid["total_doses"] = df_obito_por_covid[["DOSE_1_COV","DOSE_2_COV","DOSE_REF"]].count(axis=1)
# df_obito_por_covid

total_mortes_dose = len(df_obito_por_covid.loc[df_obito_por_covid["total_doses"] == 3]) + \
    len(df_obito_por_covid.loc[df_obito_por_covid["total_doses"] == 2]) + \
    len(df_obito_por_covid.loc[df_obito_por_covid["total_doses"] == 1]) + \
    len(df_obito_por_covid.loc[df_obito_por_covid["total_doses"] == 0])

In [14]:
labels_doses = []
values_doses = []
labels_doses.append("0")
values_doses.append(len(df_obito_por_covid.loc[df_obito_por_covid["total_doses"] == 0]))  
labels_doses.append("1")
values_doses.append(len(df_obito_por_covid.loc[df_obito_por_covid["total_doses"] == 1]))
labels_doses.append("2")
values_doses.append(len(df_obito_por_covid.loc[df_obito_por_covid["total_doses"] == 2]))
labels_doses.append("3")
values_doses.append(len(df_obito_por_covid.loc[df_obito_por_covid["total_doses"] == 3]))


print("doses\t", "óbitos")
for index in range(len(labels_doses)):
    print(f"{labels_doses[index]} doses\t", values_doses[index])

total_mortes_dose = sum(values_doses)
print("total\t", total_mortes_dose)

doses	 óbitos
0 doses	 14272
1 doses	 2481
2 doses	 19113
3 doses	 10824
total	 46690


In [15]:


# data = {"labels": labels_doses, "values": values_doses}
# fig = px.bar(df, y=values_doses, x=labels_doses, 
#            title="Óbitos por doses recebidas")
# fig.update_layout(xaxis_title='Doses', yaxis_title='Óbitos')
# fig.show()

fig = go.Figure(data=[go.Bar(
    x= labels_doses,
    y= values_doses,
    hovertemplate = '%{x} doses %{y:s}',
)])
fig.update_layout(title_text='Óbitos por doses recebidas')
fig.update_layout(xaxis_title='óbitos', yaxis_title="doses recebidas")

## filtrar por obito por COVID e levando conta a idade

## por idade

In [16]:
# selecionar apenas casos de óbitos pela COVID para individuos com menos de <19
print("idade\t", "óbitos")
for idade in range(0, 19):
    df_menor_de_x_idade = df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] <= idade)]
    print(f"{idade} anos\t", len(df_menor_de_x_idade))

idade	 óbitos
0 anos	 13
1 anos	 105
2 anos	 169
3 anos	 215
4 anos	 256
5 anos	 292
6 anos	 315
7 anos	 343
8 anos	 367
9 anos	 396
10 anos	 425
11 anos	 458
12 anos	 469
13 anos	 479
14 anos	 493
15 anos	 512
16 anos	 539
17 anos	 559
18 anos	 584


## Faixa Etária


In [19]:
labels_faixa = []
values_faixa = []

values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 0) & (df["NU_IDADE_N"] <= 4)]))
labels_faixa.append("0-4")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 5) & (df["NU_IDADE_N"] <= 9)]))
labels_faixa.append("5-9")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 10) & (df["NU_IDADE_N"] <= 14)]))
labels_faixa.append("10-14")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 15) & (df["NU_IDADE_N"] <= 19)]))
labels_faixa.append("15-19")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 20) & (df["NU_IDADE_N"] <= 24)]))
labels_faixa.append("20-24")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 25) & (df["NU_IDADE_N"] <= 29)]))
labels_faixa.append("25-29")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 30) & (df["NU_IDADE_N"] <= 34)]))
labels_faixa.append("30-34")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 35) & (df["NU_IDADE_N"] <= 39)]))
labels_faixa.append("35-39")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 40) & (df["NU_IDADE_N"] <= 44)]))
labels_faixa.append("40-44")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 45) & (df["NU_IDADE_N"] <= 49)]))
labels_faixa.append("45-49")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 50) & (df["NU_IDADE_N"] <= 54)]))
labels_faixa.append("50-54")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 55) & (df["NU_IDADE_N"] <= 59)]))
labels_faixa.append("55-59")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 60) & (df["NU_IDADE_N"] <= 64)]))
labels_faixa.append("60-64")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 65) & (df["NU_IDADE_N"] <= 69)]))
labels_faixa.append("65-69")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 70) & (df["NU_IDADE_N"] <= 74)]))
labels_faixa.append("70-74")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 75) & (df["NU_IDADE_N"] <= 79)]))
labels_faixa.append("75-79")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 80) & (df["NU_IDADE_N"] <= 84)]))
labels_faixa.append("80-84")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 85) & (df["NU_IDADE_N"] <= 89)]))
labels_faixa.append("85-89")
values_faixa.append(len(df.loc[(df["EVOLUCAO"] == 2) & (df["CLASSI_FIN"]==5) & (df["NU_IDADE_N"] >= 90)]))
labels_faixa.append("90+")

print("faixa etária\t", "óbitos")
for index in range(len(labels_faixa)):
    print(f"{labels_faixa[index]}\t", values_faixa[index])


faixa etária	 óbitos
0-4	 256
5-9	 140
10-14	 97
15-19	 116
20-24	 206
25-29	 290
30-34	 409
35-39	 590
40-44	 796
45-49	 1098
50-54	 1524
55-59	 2334
60-64	 3298
65-69	 4228
70-74	 5283
75-79	 6213
80-84	 6700
85-89	 6354
90+	 6758


In [18]:
fig = go.Figure(go.Bar(
            x=values_faixa,
            y=labels_faixa,
            orientation='h',
            hovertemplate = 'faixa %{y} anos<br>óbitos: %{x:s}',
))
fig.update_layout(xaxis_title='óbitos', yaxis_title="faixa etária")
fig.show()